Library Import

In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile,BasicAer
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator
from qiskit.algorithms.optimizers import SPSA
import pandas as pd
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer


# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")
from pylab import cm
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [2]:
class QKTCallback:
    """Callback wrapper class."""

    def __init__(self) -> None:
        self._data = [[] for i in range(5)]

    def callback(self, x0, x1=None, x2=None, x3=None, x4=None):
        """
        Args:
            x0: number of function evaluations
            x1: the parameters
            x2: the function value
            x3: the stepsize
            x4: whether the step was accepted
        """
        self._data[0].append(x0)
        self._data[1].append(x1)
        self._data[2].append(x2)
        self._data[3].append(x3)
        self._data[4].append(x4)

    def get_callback_data(self):
        return self._data

    def clear_callback_data(self):
        self._data = [[] for i in range(5)]

Preparing the Dataset

In [3]:
df = pd.read_csv(r'prefinaloutput.csv')
ytrain= df['isFraud']
Xtrain= df.drop(['isFraud'],axis=1)
Xtrain

#To reduce training dataset size
Xtrain,_ , ytrain, _ = train_test_split(Xtrain, ytrain,stratify=ytrain, test_size=0.5)

Traceback (most recent call last):
  Cell In[3], line 7
    Xtrain,_ , ytrain, _ = train_test_split(Xtrain, ytrain,stratify=ytrain, test_size=0.5)
NameError: name 'train_test_split' is not defined

Use %tb to get the full traceback.


In [5]:
dft = pd.read_csv(r'testfinal.csv')
ytest= dft['isFraud']
Xtest= dft.drop(['isFraud'],axis=1)
ytest

0      1
1      1
2      1
3      1
4      1
      ..
494    0
495    0
496    0
497    0
498    0
Name: isFraud, Length: 499, dtype: int64

Define the Quantum Feature Map


In [6]:
training_params = ParameterVector("θ", 1)
fm0 = QuantumCircuit(17)
for i in range(17):
    fm0.ry(training_params[0], i)


# Use ZZFeatureMap to represent input data
fm1 = ZZFeatureMap(17,entanglement='linear')

# Create the feature map, composed of our two circuits
fm = fm0.compose(fm1)

print(circuit_drawer(fm, output='text'))
print(f"Trainable parameters: {training_params}")

      ┌──────────┐»
 q_0: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_1: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_2: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_3: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_4: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_5: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_6: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_7: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_8: ┤ Ry(θ[0]) ├»
      ├──────────┤»
 q_9: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_10: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_11: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_12: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_13: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_14: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_15: ┤ Ry(θ[0]) ├»
      ├──────────┤»
q_16: ┤ Ry(θ[0]) ├»
      └──────────┘»
«      ┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
« q_0: ┤0                                                                                                            ├
«      │                                                      

In [8]:
# Instantiate quantum kernel
quant_kernel = TrainableFidelityQuantumKernel(feature_map=fm, training_parameters=training_params)

# Set up the optimizer
cb_qkt = QKTCallback()
spsa_opt = SPSA(maxiter=10, callback=cb_qkt.callback, learning_rate=0.05, perturbation=0.05)

# Instantiate a quantum kernel trainer.
qkt = QuantumKernelTrainer(
    quantum_kernel=quant_kernel, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi / 2]
)

In [ ]:
# Train the kernel using QKT directly
qka_results = qkt.fit(Xtrain, ytrain)
optimized_kernel = qka_results.quantum_kernel
print(qka_results)

Fit and Test the Model

In [ ]:
# Use QSVC for classification
qsvc = QSVC(quantum_kernel=optimized_kernel)

# Fit the QSVC
qsvc.fit(Xtrain, ytrain)

# Predict the labels
labels_test = qsvc.predict(Xtest)

# Evalaute the test accuracy
accuracy_test = metrics.balanced_accuracy_score(y_true=ytest, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")

Visualize the Kernel Training Process

In [ ]:
plot_data = cb_qkt.get_callback_data()  # callback data
K = optimized_kernel.evaluate(X_train)  # kernel matrix evaluated on the training samples

plt.rcParams["font.size"] = 20
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].plot([i + 1 for i in range(len(plot_data[0]))], np.array(plot_data[2]), c="k", marker="o")
ax[0].set_xlabel("Iterations")
ax[0].set_ylabel("Loss")
ax[1].imshow(K, cmap=cm.get_cmap("bwr", 20))
fig.tight_layout()
plt.show()